# 一、Modularity定义
<img src="pic/Louvain_1.png" width="500" height="500">
&nbsp;&nbsp;&nbsp;&nbsp; 定义社区内边的权重$e_c = \frac{1}{2}∑_{(u∈c)}∑_{(v∈c)}A_{uv}$，其中c为社区，即就是子图邻接矩阵的元素和。
<img src="pic/Louvain_2.png" width="500" height="500">
<img src="pic/Louvain_3.png" width="500" height="500">
&nbsp;&nbsp;&nbsp;&nbsp; 定义社区边的比例为$F = \frac{\sum_{c \in C}e_c}{\frac{1}{2}\sum_u\sum_v A_{uv}}$，其中分子为全部社区的社区内边的权重和，而分母m为整个大图的邻接矩阵元素和。注意，<b>社区间的边权重包含在分母而不包含在分子</b>。最优问题，就是最大化“社区边权重比例”。
&nbsp;&nbsp;&nbsp;&nbsp;但现在问题是，整个公式肯定倾向于让整个大图成为一个社区，这样就没有社区之间的节点了。现在转换思路，将使社区间的边最少，<b>转变为使社区间的边少于预期</b>。现在定义整个随机基线：<br>
<img src="pic/Louvain_4.png" width="500" height="500"><br>
&nbsp;&nbsp;&nbsp;&nbsp; 什么样的随机图具有可比性呢？我们给出可比较的三点定义：①具有相同的节点数；②具有相同的边数；③所有节点的度数一样。OK，那对于一个社区（观察社区内的边和社区向社区外的边），对于节点u和v，其边分别为$K_u$与$K_v$，则全部的节点对为$K_u*K_v$，那刚好发生给定社区连接情况的概率是多少呢？对于大图的邻接矩阵，共有$\frac{1}{2m-1}$的可能刚好连接到给定的节点边，其中m与之前一样是整个大图的邻接矩阵元素和，则随机基线为$E=\frac{K_u*K_v}{2m-1}$。最后可以得到$Q=\frac{1}{2m}\sum_{c \in C}(\sum_{u \in c}\sum_{v \in c}(A_{uv}-\frac{k_u*k_v}{2m-1}))=\frac{1}{2m}\sum_{c \in C}(\sum_{u \in c}\sum_{v \in c}(A_{uv}-\frac{k_u*k_v}{2m}))=\frac{1}{2m}\sum_{c \in C}(\sum_{u \in c}\sum_{v \in c}A_{uv}-\frac{(\sum_{v \in c}K_v)^2}{2m})$。

# 二、算法流程
1. 初始化，每个节点单独为一个社区。
2. 计算每个节点合并后的∆Q，合并最大∆Q，迭代进行，直到没有节点能够合并。
3. 聚合社区，将地②步的每个社区聚合为一个节点，内部的连接用一个自连接表示，外部的多条连接也合并到一起。
<img src="pic/Louvain_5.png" width="500" height="500"> ==>
<img src="pic/Louvain_6.png" width="500" height="500">
4. 重复②③。


In [1]:
import numpy as np

In [ ]:
def Q(graph, idx, m):
    return np.sum(graph[idx, idx]) - np.sum(graph[:, idx])**2/m

def setp1(graph):
    n = len(graph)
    m = np.sum(graph)
    community = np.eye(n) # 初始化社区
    communityQ = np.sum(graph*community, axis=1) - np.sum(graph*community, axis=1)**2/m # 初始化每个社区的Q

    flag = True
    # 1. step1
    while flag:
        flag = False
        for i in range(n):
            max_Q = 0
            best_C = None
            for j in range(n):
                if community[i,j]:
                    continue
                subC_1 = community[j, :] | np.eye(1, n, k=i, dtype=bool)[0]
                subC_2 = community[i, :] ^ np.eye(1, n, k=i, dtype=bool)[0]
                Q_new = Q(graph, subC_1, m) + Q(graph, subC_2, m)
                Q_old = np.sum(communityQ[[i,j]])
                if Q_new > Q_old:
                    max_Q = Q_new
                    best_C = j
            if max_Q > 0:
                # 更新社区
                subC_1 = community[best_C, :] | np.eye(1, n, k=i, dtype=bool)[0]
                community[subC_1, subC_1] = 1
                subC_2 = community[i, :] ^ np.eye(1, n, k=i, dtype=bool)[0]
                community[subC_2, subC_2] = 0
                # 更新社区Q
                communityQ[subC_1] = Q(graph, subC_1, m)
                communityQ[subC_2] = Q(graph, subC_2, m)
                flag = True
    return community

def setp2(graph, community):

    def get_communities_from_matrix(community):
        """从n×n社区矩阵获取社区列表"""
        n = len(community)
        visited = np.zeros(n, dtype=bool)
        communities = []
        for i in range(n):
            if not visited[i]:
                # 找到与i在同一社区的所有节点
                members = np.where(community[i] == 1)[0]
                communities.append(list(members))
                visited[members] = True

        return communities

    def rebuild_graph(graph, communities):
        """
        根据社区列表重构新图
        graph: 上一轮邻接矩阵 (n×n)
        communities: 社区列表，如 [[0,1,2], [3,4,5]]
        返回：
        new_graph: 新邻接矩阵 (k×k)，k为社区数
        community_map: 原始节点→新节点ID的映射
        """
        k = len(communities)  # 社区数量
        n = len(graph)
        m = np.sum(graph)
        # 创建映射：原始节点→新社区ID
        node_to_community = np.zeros(n, dtype=int)
        for comm_id, comm in enumerate(communities):
            node_to_community[comm] = comm_id

        # 初始化新图
        new_graph = np.zeros((k, k))

        # 填充新图的权重（社区间的总连接权重）
        for i in range(k):
            for j in range(k):
                if i > j:  # 只计算上三角
                    continue
                weight = np.sum(graph[communities[i], communities[j]])
                new_graph[i, j] = weight
                new_graph[j, i] = weight  # 对称矩阵

        return new_graph, node_to_community

    communities = get_communities_from_matrix(community)
    new_graph, node_to_community = rebuild_graph(graph, communities)

    return new_graph, node_to_community


def Louvain(graph):
    hierarchy = []
    while True:
        # 1. step1
        community = setp1(graph)
        # 2. step2
        new_graph, node_to_community = setp2(graph, community)
        if len(new_graph) == len(graph):
            break
        hierarchy.append(node_to_community)
    return hierarchy
